In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [3]:
def create_url(keyword, max_results = 100):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
#                     'start_time': start_date,
#                     'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'author_id,context_annotations,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,source,text,withheld',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [4]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [5]:
#Inputs for the request
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAL1CVgEAAAAALQLs8LPJXu2qbtDpVNY8T%2BUZp30%3DrnadD6NJADffq4zIIdECRUHZk7unRPPShL8cigoKKUR9nkjsJQ"
bearer_token = "Apna use karo"
headers = create_headers(bearer_token)
keyword = "covid lang:en"
max_results = 100

url = create_url(keyword, max_results)

In [18]:
# next_token = None
i = 0
for i in range(100):
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    next_token = json_response['meta']['next_token']
    df_data = pd.DataFrame(json_response['data'])
    df_user = pd.DataFrame(json_response['includes']['users'])
    if i == 0:
        df_data.to_csv('./data/out-8.csv')
        df_user.to_csv('./data/user-8.csv')
    else:
        df_data.to_csv('./data/out-8.csv', mode='a', header=False)
        df_user.to_csv('./data/user-8.csv', mode='a', header=False)
    print(i)

Endpoint Response Code: 429


Exception: (429, '{"title":"Too Many Requests","detail":"Too Many Requests","type":"about:blank","status":429}')

In [19]:
next_token

'b26v89c19zqg8o3fpdy6ertnx67qqyh9tbfqql4g3u9vh'